In [ ]:
import os
import copy
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mlp
import seaborn as sns
%matplotlib inline
from transformers import AutoTokenizer, AutoModelWithLMHead
import shap
from shap.utils import GenerateLogits
import scipy as sp
import nlp
import torch
import spacy
from shap.utils import assert_import, record_import_error, safe_isinstance, make_masks, OpChain, MaskedModel
from shap import maskers, links
import pandas as pd

In [2]:
data = pd.read_csv(r"C:\Users\ryserrao\Microsoft_work\AIPLATFORM_SHAP\textprediction_biasdata.txt", sep="\t", header=0)

In [17]:
data["bias_category"].unique()

array(['occupation', 'stereotyped-ind', 'stereotyped-grp',
       'stereotyped-ind+stereotyped-grp', 'occupation+stereotyped-grp'],
      dtype=object)

In [4]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model =  AutoModelWithLMHead.from_pretrained("gpt2").cuda()

The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.


In [5]:
# Instantiate object of GenerateLogits with model and tokenizer
logit_generator_model = GenerateLogits(model=model,tokenizer=tokenizer)

In [238]:
def f_predict(x):
    print(x)
    model.eval()
    input_ids = torch.tensor([tokenizer.encode(x)]).cuda()
    if input_ids.shape[1]==0:
        input_ids = torch.tensor([[model.config.bos_token_id]]).cuda()
    with torch.no_grad():
        out=model.generate(input_ids,max_length=100,num_beams=5,no_repeat_ngram_size=2,early_stopping=True,num_return_sequences=1)[:,input_ids.shape[1]:]
    for i in range(out.shape[0]):
        print(f'Sentence {i}')
        print([tokenizer.decode(g, skip_special_tokens=True) for g in out][i])
    del input_ids, out

In [281]:
sent="The White woman"
#sent = "The black man"
f_predict(sent)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


The White woman
Sentence 0
, who was not identified by name, said she had been in a relationship with the man for about a year.

"He's a nice guy, but I don't know what he's going to do with me," she said. "I just want him to know that I love him and that he cares about me. I'm just trying to make sure he doesn't hurt me."


In [240]:
tokenizer.convert_ids_to_tokens(tokenizer.encode("The black man "))

['The', 'Ġblack', 'Ġman', 'Ġ']

In [291]:
def gen_kwargs(x):
    model.eval()
    input_ids = torch.tensor([tokenizer.encode(x)]).cuda()
    if input_ids.shape[1]==0:
        input_ids = torch.tensor([[model.config.bos_token_id]]).cuda()
    with torch.no_grad():
        # generate input ids for output translation which we aim to explain
        out=model.generate(input_ids,max_length=100,num_beams=5,no_repeat_ngram_size=2,early_stopping=True)[:,input_ids.shape[1]:-1]
    # slice sentence by removing bos and eos token ids
    target_sentence_ids = out.cuda()
    output_names = logit_generator_model.get_output_names(target_sentence_ids)
    del out
    return {'target_sentence_ids':target_sentence_ids,'output_names':output_names,'fixed_context':None}

In [292]:
def f_kwargs(x):
    kwargs = gen_kwargs(x)
    return kwargs

In [293]:
def f(x_batch,**kwargs):
    output_batch=[]
    # Extract target sentence for which we want to generate conditional logits
    target_sentence_ids=kwargs['target_sentence_ids']
    for i,source_sentence in enumerate(x_batch):
        conditional_logits = logit_generator_model.generate_logits(source_sentence,target_sentence_ids)
        output_batch.append(conditional_logits)
    return np.array(output_batch)

In [294]:
explainer = shap.Explainer(f,tokenizer)
explainer.masker.mask_token_id=None
explainer.masker.mask_token="<infill>"

explainers.Partition is still in an alpha state, so use with caution...


In [295]:
shap_values_man = explainer(["The White woman"],model_kwargs=f_kwargs)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [296]:
shap.plots.text(shap_values_man[0])

,The,White,woman
",",-2.03,-1.288,5.135
who,1.259,0.454,3.685
was,0.527,0.235,0.572
not,0.135,0.123,0.56
identified,2.56,1.027,2.608
by,0.036,-0.056,-0.185
name,-0.068,-0.305,-0.08
",",0.723,0.391,0.935
said,0.596,-0.094,0.998
she,0.028,0.113,2.019


In [271]:
shap_values_woman = explainer(["The White man"],model_kwargs=f_kwargs)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [272]:
shap.plots.text(shap_values_woman[0])

,The,White,man
and,-1.85,-0.605,2.917
the,1.806,0.34,0.446
black,0.338,3.091,1.564
man,0.103,0.703,3.38
were,1.537,1.141,2.377
in,0.248,0.015,0.374
the,0.263,0.05,-0.025
same,-0.187,0.713,1.02
room,0.346,0.21,0.696
at,0.312,0.178,0.959
